In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv')

X = df.drop("purchaseValue",axis=1)
Y = df["purchaseValue"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=59)

In [ ]:
X_predict = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv')

In [ ]:
import seaborn as sns
sns.heatmap(pd.concat([X_train,Y_train],axis=1).select_dtypes(include=['number']).corr(), annot=True, cmap='coolwarm', fmt=".2f")

In [ ]:
# # Lkeep = ['device.mobileDeviceBranding', 'gclIdPresent',
# #        'totals.visits','trafficSource.adwordsClickInfo.isVideoAd','pageViews',
# #        'device.mobileDeviceModel','totals.bounces','totalHits','device.isMobile','new_visits']
# Lkeep = list(df.columns)
# L7 = ['userId','sessionId','trafficSource.adwordsClickInfo.page','sessionStart','geoNetwork.region',
#       'trafficSource.adwordsClickInfo.adNetworkType','geoNetwork.city','geoNetwork.metro','trafficSource.referralPath'] 
# # They had very distributed values, or had too many not in dataset values, so removed them, checked each of them by doing value_counts()

# for j in L7:
#     Lkeep.remove(j)
# # ['purchaseValue', 'browser', 'trafficSource.adContent', 'trafficSource.keyword', 'geoCluster', 'trafficSource.adwordsClickInfo.slot', 'userId', 'trafficSource.campaign', 'geoNetwork.networkDomain', 'gclIdPresent', 'sessionNumber', 'geoNetwork.region', 'trafficSource', 'sessionId', 'os', 'geoNetwork.subContinent', 'trafficSource.medium', 'locationCountry', 'trafficSource.adwordsClickInfo.adNetworkType', 'geoNetwork.city', 'trafficSource.adwordsClickInfo.page', 'geoNetwork.metro', 'pageViews', 'trafficSource.referralPath', 'date', 'deviceType', 'userChannel', 'totalHits', 'sessionStart', 'geoNetwork.continent', 'device.isMobile']

# for i in df.columns:
#     if len(df[i].value_counts())==1:
#         if i in Lkeep:
#             print(i)
#             Lkeep.remove(i)
# Lkeep.remove('purchaseValue')
# # print(Lkeep)

In [ ]:
# X_train = X_train[Lkeep]
# X_test = X_test[Lkeep]
# X_predict = X_predict[Lkeep]

In [ ]:
sns.heatmap(pd.concat([X_train,Y_train],axis=1).select_dtypes(include=['number']).corr(), annot=True, cmap='coolwarm', fmt=".2f")

In [ ]:
# print(X_train.info())

##################


browser -> One hot encoding maybe? but too many browsers are there, but it does matter quite a bit
maybe i could just do for the popular browsers

trafficSource.adContent -> might be null and string based

trafficSource.keyword -> might be null and string based

geoCluster -> divides into 5 regions and is not null

trafficSource.adwordsClickInfo.slot -> 3 different possible values

trafficSource.campaign -> String based and not null

geoNetwork.networkDomain -> 3 possible domains, not null, do one hot encoding

sessionNumber -> not null , has a unexpected correlation with purchaseValue

trafficSource -> not null, string based, too many values

os -> around 20 not null, could help

geoNetwork.subContinent -> zone wise continent, not null, string, will help

trafficSource.medium -> medium for reaching, not null, string, around 7 distinct values

locationCountry -> String not null, too many distinct but will help for sure

pageViews -> Integer not null

deviceType -> oneHotEncoding, only 3 values present not null

userChannel -> not null 8 distinct values

totalHits -> not null Integer

geoNetwork.continent -> Continent Not Null

device.isMobile -> True of False not null

In [ ]:
print(df['device.isMobile'].value_counts())

In [ ]:
# from sklearn.preprocessing import StandardScaler
# import pandas as pd

# def preprocess_df(df, top_n=10, scaler=None, fit_scaler=True, col_order=None, num_cols_ref=None):
#     df = df.copy()

#     str_cols = df.select_dtypes(include='object').columns
#     df[str_cols] = df[str_cols].fillna("Missing")
#     df['device.isMobile'] = df['device.isMobile'].astype(int)

#     for col in ['trafficSource', 'trafficSource.campaign']:
#         freq = df[col].value_counts().to_dict()
#         df[col + "_freq"] = df[col].map(freq)

#     for col in ['browser', 'os']:
#         top = df[col].value_counts().nlargest(top_n).index
#         for cat in top:
#             df[f"{col}_{cat}"] = (df[col] == cat).astype(int)
#         df[f"{col}_Other"] = (~df[col].isin(top)).astype(int)
#         df.drop(columns=col, inplace=True)

#     low_card = [
#         'geoNetwork.networkDomain',
#         'trafficSource.adwordsClickInfo.slot',
#         'deviceType',
#         'userChannel',
#         'trafficSource.medium',
#         'geoNetwork.subContinent',
#         'geoNetwork.continent',
#         'geoCluster'
#     ]
#     df = pd.get_dummies(df, columns=low_card, prefix=low_card)

#     if col_order is not None:
#         df = df.reindex(columns=col_order, fill_value=0)

#     if num_cols_ref is None:
#         num_cols = df.select_dtypes(include=['int64', 'float64']).columns
#     else:
#         num_cols = num_cols_ref

#     if scaler is None and fit_scaler:
#         scaler = StandardScaler()
#         df[num_cols] = scaler.fit_transform(df[num_cols])
#     elif scaler is not None:
#         df[num_cols] = scaler.transform(df[num_cols])

#     if col_order is None:
#         col_order = df.columns.tolist()
#     if num_cols_ref is None:
#         num_cols_ref = num_cols.tolist()

#     return df, scaler, col_order, num_cols_ref


In [ ]:
# for df in [X_train, X_test, X_predict]:
#     df['date'] = pd.to_datetime(df['date'], errors='coerce')
# for df in [X_train, X_test, X_predict]:
#     df['year'] = df['date'].dt.year
#     df['month'] = df['date'].dt.month
#     df['day'] = df['date'].dt.day
#     df['dayofweek'] = df['date'].dt.dayofweek
#     df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
#     df['quarter'] = df['date'].dt.quarter
#     df['dayofyear'] = df['date'].dt.dayofyear
# for df in [X_train, X_test, X_predict]:
#     df.drop('date', axis=1, inplace=True)
# for df in [X_train, X_test, X_predict]:
#     df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
#     df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
#     df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
#     df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import FunctionTransformer
# import numpy as np
# import pandas as pd

# def replace_true_false(X):
#     return np.where(X == True, 1, np.where(X == False, -1, X))

# categorical_cols = ['geoNetwork.subContinent', 'geoCluster']

# for df in [X_train, X_test, X_predict]:
#     df.drop('sessionNumber', inplace=True, axis=1) # ise comment out krke bhi try krna, sometimes its better

# numeric_cols = [col for col in X_train.columns if col not in categorical_cols]

# preprocess = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
#     ],
#     remainder='passthrough'
# )

# X_train_enc = preprocess.fit_transform(X_train)
# X_test_enc = preprocess.transform(X_test)
# X_predict_enc = preprocess.transform(X_predict)

# encoded_cat_cols = preprocess.named_transformers_['cat'].get_feature_names_out(categorical_cols)
# all_columns = list(encoded_cat_cols) + numeric_cols

# X_train_enc = pd.DataFrame(X_train_enc, columns=all_columns, index=X_train.index)
# X_test_enc = pd.DataFrame(X_test_enc, columns=all_columns, index=X_test.index)
# X_predict_enc = pd.DataFrame(X_predict_enc, columns=all_columns, index=X_predict.index)

# pipeline = Pipeline(steps=[('replace_true_false', FunctionTransformer(replace_true_false, validate=False))])

# X_train_pre = pipeline.transform(X_train_enc)
# X_test_pre = pipeline.transform(X_test_enc)
# X_predict_pre = pipeline.transform(X_predict_enc)

# X_train_pre = pd.DataFrame(X_train_pre, columns=all_columns, index=X_train.index)
# X_test_pre = pd.DataFrame(X_test_pre, columns=all_columns, index=X_test.index)
# X_predict_pre = pd.DataFrame(X_predict_pre, columns=all_columns, index=X_predict.index)

# for col in X_train_pre.columns:
#     try:
#         X_train_pre[col] = X_train_pre[col].astype(int)
#         X_test_pre[col] = X_test_pre[col].astype(int)
#         X_predict_pre[col] = X_predict_pre[col].astype(int)
#         # X_test_pre[col] = pd.to_numeric(X_test_pre[col], errors='coerce')
#         # X_predict_pre[col] = pd.to_numeric(X_predict_pre[col], errors='coerce')
#     except:
#         pass
        
# X_train_pre = X_train_pre.select_dtypes(include=['number'])
# X_test_pre = X_test_pre.select_dtypes(include=['number'])
# X_predict_pre = X_predict_pre.select_dtypes(include=['number'])


In [ ]:
# X_train_pre.shape

In [ ]:
# X_train_pre.head()

In [ ]:
# X_train_pre.columns

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer,KNNImputer
# from sklearn.preprocessing import OneHotEncoder,FunctionTransformer
# from sklearn.compose import ColumnTransformer

# def replace_true_false(X):
#     return np.where(X == True, 1, np.where(X == False, 0, X))

# pipeline = Pipeline(steps=[
#     ('replace_true_false', FunctionTransformer(replace_true_false, validate=False)),
#     ('imputer', KNNImputer(n_neighbors=5))
# ])

In [ ]:
# pipeline.fit(X_train)

In [ ]:
# X_train = pipeline.transform(X_train)
# X_test = pipeline.transform(X_test)
# X_predict = pipeline.transform(X_predict)

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# # model = LinearRegression()
# # rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=10, min_samples_leaf=5, random_state=42)
# # rf_model.fit(X_train,Y_train)
# # from sklearn.linear_model import LinearRegression
# # from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
# # from sklearn.ensemble import RandomForestRegressor

# # rf_model = RandomForestRegressor()

# # param_grid = {
# #     'n_estimators': [100, 200, 300],
# #     'max_depth': [5, 10, 20, None],
# #     'min_samples_split': [2, 10, 20],
# #     'min_samples_leaf': [1, 2, 4]
# # }

# # random_search = RandomizedSearchCV(
# #     estimator=rf_model,
# #     param_distributions=param_grid,
# #     n_iter=10,
# #     cv=2,
# #     scoring='r2',
# #     n_jobs=-1,
# #     verbose=1
# # )

# # random_search.fit(X_train, Y_train)
# # rf_model = random_search.best_estimator_

# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import RidgeCV
# from sklearn.preprocessing import PolynomialFeatures

# model = Pipeline([
#     ('scaler', StandardScaler()),
#     ('poly',PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
#     ('ridge', RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0, 100.0], cv=5))
# ])
# model.fit(X_train, Y_train)


In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import FunctionTransformer, StandardScaler
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.impute import SimpleImputer,KNNImputer
# from sklearn.linear_model import Ridge,LinearRegression,Lasso
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.ensemble import HistGradientBoostingRegressor
# from lightgbm import LGBMRegressor
# import xgboost as xgb
# from xgboost import XGBRegressor

# # text_cols = [
# #     'trafficSource.adContent',
# #     'trafficSource.keyword',
# #     'trafficSource.campaign',
# #     'trafficSource',
# #     'locationCountry'
# # ]
# # num_cols = [c for c in X_train_pre.columns if c not in text_cols]

# # flatten = FunctionTransformer(lambda X: X.ravel(), validate=False)

# # preprocessor = ColumnTransformer([
# #     *[
# #         (
# #             f"tfidf_{col}",
# #             Pipeline([
# #                 ('impute', SimpleImputer(strategy='constant', fill_value='')),
# #                 ('flatten', flatten),
# #                 ('tfidf', TfidfVectorizer(max_features=100, stop_words='english'))
# #             ]),
# #             [col]
# #         )
# #         for col in text_cols
# #     ],
# #     (
# #         'num',
# #         Pipeline([
# #             ('impute', SimpleImputer(strategy='mean')),
# #             ('scale',   StandardScaler())
# #         ]),
# #         num_cols
# #     )
# # ], remainder='drop')
# from sklearn.linear_model import RidgeCV

# pipeline = Pipeline([
#     # ('preproc', preprocessor),
#     # ('to_dense', FunctionTransformer(lambda X: X.toarray(), accept_sparse=True)),
#     ('impute',SimpleImputer(strategy='mean')),
#     # ('imputer', KNNImputer(n_neighbors=5)),
#     ('scale',StandardScaler()),
#     # ('model', Ridge())
#     ('poly',PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
#     # ('model', Lasso(alpha=0.1, max_iter=1000))
#     # ('model',RandomForestRegressor(n_estimators=100,random_state=200,n_jobs=-1)),
#     # ('reg', HistGradientBoostingRegressor(
#     #     max_iter=200,
#     #     learning_rate=0.05,
#     #     max_depth=2,
#     #     min_samples_leaf=10,
#     #     l2_regularization=2.0,
#     #     # random_state=22,
#     #     # verbose=1
#     # ))
#     ('ridge', RidgeCV(alphas=[0.01], cv=10))
#     # ('model', XGBRegressor(objective='reg:squarederror', max_depth=4, learning_rate=0.1, n_estimators=100, eval_metric='rmse', use_label_encoder=False))
#     # ('lgbm', LGBMRegressor(
#     #     n_estimators=1000,
#     #     learning_rate=0.05,
#     #     max_depth=6,
#     #     random_state=42,
#     #     n_jobs=-1
#     # ))

# ])


# pipeline.fit(X_train_pre, Y_train)

# from sklearn.metrics import r2_score

# print(r2_score(Y_test,pipeline.predict(X_test_pre)))
# print(r2_score(Y_train,pipeline.predict(X_train_pre)))

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# from sklearn.impute import SimpleImputer
# from sklearn.linear_model import RidgeCV, Ridge
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import r2_score

# pipeline = Pipeline([
#     ('impute', SimpleImputer(strategy='mean')),
#     ('scale', StandardScaler()),
#     ('poly', PolynomialFeatures(interaction_only=True,include_bias=False)),
#     ('ridge', Ridge())
# ])

# param_grid = {
#     'poly__degree': [2],
#     'ridge__alpha': [0.01, 0.1, 1.0, 10, 100]
# }

# grid = GridSearchCV(
#     pipeline,
#     param_grid,
#     scoring='r2',
#     cv=5,
#     n_jobs=-1,
#     verbose=1
# )

# grid.fit(X_train_pre, Y_train)
# pipeline = grid
# print("Best Params:", grid.best_params_)
# print("Best R2 on Training Set:", grid.best_score_)
# print("Test R2:", r2_score(Y_test, grid.predict(X_test_pre)))
# print("Train R2:", r2_score(Y_train, grid.predict(X_train_pre)))


In [ ]:
# xgb_model = XGBRegressor(
#     objective='reg:squarederror',
#     max_depth=3,
#     learning_rate=0.05,
#     n_estimators=500,
#     eval_metric='rmse',
#     use_label_encoder=False
# )

# xgb_model.fit(
#     X_train_pre, Y_train,
#     eval_set=[(X_test_pre, Y_test)],
#     early_stopping_rounds=20,
#     verbose=False
# )

# print("Train R2:", r2_score(Y_train, xgb_model.predict(X_train_pre)))
# print("Test R2:", r2_score(Y_test, xgb_model.predict(X_test_pre)))

In [ ]:
import pandas as pd
def preprocess(df, median_pageViews=None, ohe_model=None, categorical_cols_to_encode=None, fit_encoder=False,tfidf=None):
    df = df.copy()

    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='coerce')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day_of_week'] = df['date'].dt.dayofweek
    df = df.drop('date', axis=1)

    df['totals.bounces'] = df['totals.bounces'].fillna(0)
    df['new_visits'] = df['new_visits'].fillna(0)

    if median_pageViews is not None:
        df['pageViews'] = df['pageViews'].fillna(median_pageViews)
    else:
        df['pageViews'] = df['pageViews'].fillna(df['pageViews'].median())

    df['trafficSource.isTrueDirect'] = df['trafficSource.isTrueDirect'].astype(str).replace(
        {'True': True, 'False': False, 'nan': False}
    ).astype(bool)

    categorical_fill_cols = {
        'trafficSource.keyword': 'unknown', 'trafficSource.adContent': 'unknown_ad_content',
        'trafficSource.adwordsClickInfo.adNetworkType': 'unknown_ad_network',
        'trafficSource.adwordsClickInfo.slot': 'unknown_slot',
        'trafficSource.referralPath': 'unknown_referral_path',
        'trafficSource': 'unknown_traffic_source', 'trafficSource.campaign': 'unknown_campaign'
    }

    df['trafficSource.adwordsClickInfo.isVideoAd'] = df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True)
    
    df['trafficSource.adwordsClickInfo.isVideoAd'] = (
        df['trafficSource.adwordsClickInfo.isVideoAd']
        .astype(str)
        .replace({'True': True, 'False': False, 'nan': True})
        .astype(bool)
        .astype(int)
    )
    
    for col, fill_value in categorical_fill_cols.items():
        if col in df.columns:
            df[col] = df[col].fillna(fill_value)

    Lvec = ['trafficSource.keyword','trafficSource.referralPath','trafficSource.adContent','trafficSource.campaign','trafficSource.adwordsClickInfo.adNetworkType']

    for i in Lvec:
        df[i] = df[i].fillna('(not provided)')
        df[i] = df[i].replace('/',' ',regex=False)
        df[i] = df[i].str.lower().str.strip()

    
    from sklearn.feature_extraction.text import TfidfVectorizer
    if not tfidf:
        tfidf = TfidfVectorizer(max_features=100, stop_words='english')
        tfidf.fit(df[Lvec])
    keyword_tfidf = tfidf.transform(df[Lvec])
    keyword_tfidf_df = pd.DataFrame(keyword_tfidf.toarray(), columns=tfidf.get_feature_names_out(), index=df.index)
    df = pd.concat([df, keyword_tfidf_df], axis=1)
    
    df.drop(columns=Lvec, inplace=True)
    
    cols_to_drop = [
        'device.browserSize', 'device.browserVersion', 'device.flashVersion', 'device.language',
        'device.mobileDeviceBranding', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel',
        'device.mobileInputSelector', 'device.operatingSystemVersion', 'device.screenColors',
        'device.screenResolution', 'geoNetwork.city', 'geoNetwork.metro', 'geoNetwork.networkDomain',
        'geoNetwork.networkLocation', 'geoNetwork.region', 'socialEngagementType',
        'trafficSource.adwordsClickInfo.page', 'sessionId', 'sessionStart', 'userId',
        'locationZone', 'browserMajor', 'screenSize', 'gclIdPresent','trafficSource.adwordsClickInfo.'
    ]
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors='ignore')

    if categorical_cols_to_encode is not None and len(categorical_cols_to_encode) > 0:
        if fit_encoder:
            from sklearn.preprocessing import OneHotEncoder
            ohe_model = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
            transformed_data = ohe_model.fit_transform(df[categorical_cols_to_encode])
        else:
            transformed_data = ohe_model.transform(df[categorical_cols_to_encode])
        
        ohe_df = pd.DataFrame(transformed_data,
                              columns=ohe_model.get_feature_names_out(categorical_cols_to_encode),
                              index=df.index)
        df = pd.concat([df.drop(columns=categorical_cols_to_encode), ohe_df], axis=1)

    return df, ohe_model,tfidf


In [ ]:
categorical_cols = [
    'userChannel','geoCluster','trafficSource','geoNetwork.subContinent','locationCountry','geoNetwork.continent','deviceType','os','browser','trafficSource.medium'
]

X_train_pre, ohe_model,tdf = preprocess(X_train, categorical_cols_to_encode=categorical_cols, fit_encoder=True)
median_pageViews = X_train['pageViews'].median()
X_test_pre, _,_ = preprocess(X_test, median_pageViews=median_pageViews,ohe_model=ohe_model, categorical_cols_to_encode=categorical_cols, fit_encoder=False,tfidf=tdf)
X_predict_pre, _,_ = preprocess(X_predict, median_pageViews=median_pageViews,ohe_model=ohe_model, categorical_cols_to_encode=categorical_cols, fit_encoder=False,tfidf=tdf)


In [ ]:
for i in X_train_pre.columns:
    if 'userChannel' in i:
        print(i)

In [ ]:
len(list(X_predict_pre.columns))

In [ ]:
import re

X_train_pre.columns = [re.sub(r'[\"\{\}\[\]\:\n\r\t\\]', '_', col) for col in X_train_pre.columns]
X_test_pre.columns = X_train_pre.columns
X_predict_pre.columns = X_train_pre.columns

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_pre = scaler.fit_transform(X_train_pre)
X_test_pre = scaler.transform(X_test_pre)
X_predict_pre = scaler.transform(X_predict_pre)


In [ ]:
import lightgbm as lgb
lgbm = lgb.LGBMRegressor(
    objective='regression_l1',
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,          
    max_depth=-1, 
    min_child_samples=20,  
    subsample=0.8,     
    colsample_bytree=0.8,     
    reg_alpha=0.1,   
    reg_lambda=0.1,
    random_state=42,
    n_jobs=-1
)
lgbm.fit(X_train_pre,Y_train)
model = lgbm

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42
)
xgb.fit(X_train_pre, Y_train)
model = xgb

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_pre, Y_train)
model = ridge

In [ ]:
from sklearn.linear_model import ElasticNet
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic.fit(X_train_pre, Y_train)
model = elastic

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train_pre, Y_train)
model = rf

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor(n_estimators=100, max_depth=10, random_state=42)
et.fit(X_train_pre, Y_train)
model = et

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(Y_train,model.predict(X_train_pre)))
print(r2_score(Y_test,model.predict(X_test_pre)))

In [ ]:
Y_predict = model.predict(X_predict_pre)
Y_predict = np.maximum(Y_predict, 0)

In [ ]:
Y = pd.DataFrame({'id': range(len(Y_predict)), 'purchaseValue': Y_predict})

In [ ]:
Y

In [ ]:
Y.to_csv("/kaggle/working/submission.csv",index=False)